# Federated Shap
Inspired by SHAP: https://github.com/slundberg/shap

Kernel SHAP is a method that uses a special weighted linear regression to compute the importance of each feature. The computed importance values are Shapley values from game theory and also coefficents from a local linear regression.

In [7]:
import scipy.special 
import numpy as np
import itertools

def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def shapley_kernel(M,s):
    if s == 0 or s == M:
        return 10000
    return (M-1)/(scipy.special.binom(M,s)*s*(M-s))

#Function that imitates the model, takes in instance features and outputs predictons
def f(X):
    np.random.seed(0)
    beta = np.random.rand(X.shape[-1])
    return np.dot(X,beta) + 10

#Original shap function
'''
    f: model
    x: one instance with features
    reference: To determine the impact
        of a feature, that feature is set to "missing" and the change in the model output
        is observed. Since most models aren't designed to handle arbitrary missing data at test
        time, we simulate "missing" by replacing the feature with the values it takes in the
        background dataset. So if the background dataset is a simple sample of all zeros, then
        we would approximate a feature being missing by setting it to zero. For small problems
        this background dataset can be the whole training set, but for larger problems consider
        using a single reference value or using the kmeans function to summarize the dataset.
    M: number of features
'''
def kernel_shap(f, x, reference, M):

    X = np.zeros((2**M,M+1))
    X[:,-1] = 1
    weights = np.zeros(2**M)
    V = np.zeros((2**M,M))
    for i in range(2**M):
        V[i,:] = reference

    ws = {}
    for i,s in enumerate(powerset(range(M))):
        s = list(s)
        V[i,s] = x[s]
        X[i,s] = 1
        ws[len(s)] = ws.get(len(s), 0) + shapley_kernel(M,len(s))
        weights[i] = shapley_kernel(M,len(s))
    y = f(V)
    tmp = np.linalg.inv(np.dot(np.dot(X.T, np.diag(weights)), X))
    return np.dot(tmp, np.dot(np.dot(X.T, np.diag(weights)), y))

#Federated Shap Function
'''
    f: model
    x: one instance with features
    reference: To determine the impact
        of a feature, that feature is set to "missing" and the change in the model output
        is observed. Since most models aren't designed to handle arbitrary missing data at test
        time, we simulate "missing" by replacing the feature with the values it takes in the
        background dataset. So if the background dataset is a simple sample of all zeros, then
        we would approximate a feature being missing by setting it to zero. For small problems
        this background dataset can be the whole training set, but for larger problems consider
        using a single reference value or using the kmeans function to summarize the dataset.
    M: number of features
    fed_pos: feature position in x after which the features are hidden and aggregated
'''
def kernel_shap_federated(f, x, reference, M, fed_pos):
    M_real = M
    M = fed_pos + 2 #with one extra feature as the aggregated hidden features
    reference_real = reference
    reference = reference_real[:fed_pos+2] #with one extra feature as the aggregated hidden features
    reference[-1] = 0 #hard to calculated reference for aggregated features, thus set it to be 0
    X = np.zeros((2**M,M+1))
    X[:,-1] = 1
    weights = np.zeros(2**M)
    V = np.zeros((2**M_real,M_real))
    for i in range(2**M_real):
        V[i,:] = reference

    ws = {}
    for i,s in enumerate(powerset(range(M))):
        s = list(s)
        V[i,s] = x[s]
        X[i,s] = 1
        ws[len(s)] = ws.get(len(s), 0) + shapley_kernel(M,len(s))
        weights[i] = shapley_kernel(M,len(s))
    y = f(V)
    tmp = np.linalg.inv(np.dot(np.dot(X.T, np.diag(weights)), X))
    return np.dot(tmp, np.dot(np.dot(X.T, np.diag(weights)), y))




In [8]:
# Dummy Testing
M = 10
np.random.seed(1)
x = np.random.randn(M)
reference = np.zeros(M)
phi = kernel_shap(f, x, reference, M)
base_value = phi[-1]
shap_values = phi[:-1]

print("  reference =", reference)
print("          x =", x)
print("shap_values =", shap_values)
print(" base_value =", base_value)
print("   sum(phi) =", np.sum(phi))
print("       f(x) =", f(x))

  reference = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
          x = [ 1.62434536 -0.61175641 -0.52817175 -1.07296862  0.86540763 -2.3015387
  1.74481176 -0.7612069   0.3190391  -0.24937038]
shap_values = [ 0.89146267 -0.43752168 -0.31836259 -0.58464256  0.3666341  -1.4865503
  0.76350731 -0.67882376  0.3074461  -0.09561896]
 base_value = 9.999999999999885
   sum(phi) = 8.727530334734068
       f(x) = 8.727530334714547


# Using Kernal Expaliner:


In [2]:
'''
import shap

explainer = shap.KernelExplainer(f, np.reshape(reference, (1, len(reference))))
shap_values = explainer.shap_values(x)
print("shap_values =", shap_values)
print("base value =", explainer.expected_value)
'''

'\nimport shap\n\nexplainer = shap.KernelExplainer(f, np.reshape(reference, (1, len(reference))))\nshap_values = explainer.shap_values(x)\nprint("shap_values =", shap_values)\nprint("base value =", explainer.expected_value)\n'

In [10]:
x[:5]

array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763])